In [ ]:
# #@title Random Code Test
# from sklearn.feature_extraction.text import CountVectorizer
# # list of text documents
# text = "The quick brown fox#  jumped! over@ the lazy. dog?" + "\n" + "i love cookies" + " Anh likes chocolate"
# print(text)
# text = [text]
# print(text[0])
# # create the transform
# # vectorizer = CountVectorizer(token_pattern=r"[\s\S]")
# vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w\w+\b|\S*[^\w\s]\S*|\s+|\r\n")
# # tokenize and build vocab
# vectorizer.fit(text)
# # summarize
# print(vectorizer.vocabulary_)
# # encode document
# vector = vectorizer.transform(text)
# # summarize encoded vector
# print(vector.shape)
# print(type(vector))
# print(vector.toarray())

oned_array = torch.as_tensor([0,1,1])
print(oned_array.shape[0])

3


In [ ]:
pip install gmpy2

     |████████████████████████████████| 3.6 MB 5.0 MB/s 


In [ ]:
pip install essential-generators

     |████████████████████████████████| 9.5 MB 5.4 MB/s 


In [ ]:
pip install bitstring

In [ ]:
#@title Import
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import torch.optim as optim
import pickle as pkl
from numpy import linalg as LA 
from essential_generators import DocumentGenerator
import hashlib
from bitstring import BitArray

In [ ]:
#@title Constant
INPUT_DIM = 1024
OUT_DIM = 32

In [ ]:
#@title Preprocess
def preprocess(file, isSentence=True):
    if isSentence:
        send_file = [file]
    else:
        send_file = open(file, "r").readlines()
        send_file = ' '.join(send_file)
        send_file = [send_file]
    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w\w+\b|\S*[^\w\s]\S*|\s+|\r\n")
    # tokenize and build vocab
    vectorizer.fit(send_file)
    # encode document
    vector = vectorizer.transform(send_file)
    # summarize encoded vector
    vector_shape = vector.shape
    doc_feat = torch.tensor(vector.toarray(), dtype=torch.float)
    #Add padding
    if vector_shape[1] < INPUT_DIM:
      pad_size = INPUT_DIM - vector_shape[1]
      padding = (0, pad_size)
      doc_feat = F.pad(doc_feat, padding, "constant", 0)
    elif vector_shape[1] < INPUT_DIM:
      pad_size = vector_shape[1] % INPUT_DIM
      padding = (0, pad_size)
      doc_feat = F.pad(doc_feat, padding, "constant", 0)
      num_chunks = doc_feat.shape[1]/INPUT_DIM
      doc_feat = doc_feat.chunk(num_chunks)
    assert doc_feat.shape[1] == INPUT_DIM, f"document input dim does not match. 1024 expected, got: {doc_feat.shape[1]}"
    return doc_feat





In [ ]:
#@title HashNN

class HashNN(nn.Module):
    def __init__(self, out_size):
        super(HashNN, self).__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512, out_size)
        self.sequential = nn.Sequential(self.fc1,
                                        nn.ReLU(),
                                        self.fc2,
                                        nn.ReLU(),
                                        self.fc3)
    def forward(self, x):
        x = self.sequential(x)
        return x


In [ ]:
x_train = np.zeros((10000, INPUT_DIM))
x_target = np.zeros(10000)
gen = DocumentGenerator() 
for i in range(10000):
    send_file = gen.sentence()
    doc_feat = preprocess(send_file)
    x_train[i] = doc_feat
    h = hashlib.new('sha256')
    hash = send_file.encode('utf-8')
    target = hashlib.sha256(hash).digest()[:4]
    bin = int.from_bytes(target, "big")
    x_target[i] = bin
    print(i)


Streaming output truncated to the last 5000 lines.
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189

In [ ]:
#@title Model training
hash_model = HashNN(1)
epochs = np.random.randint(100, 500)
optimizer = optim.SGD(hash_model.parameters(), lr=1e-5)
criterion = nn.MSELoss()
for i in range(1000):
  idx = np.random.permutation(len(x_target))
  batch_feat = torch.from_numpy(x_train[idx]).to(dtype=torch.float)
  batch_target = torch.from_numpy(x_target[idx]).to(dtype=torch.float)
  optimizer.zero_grad()
  output = hash_model(batch_feat)
  loss = criterion(output, batch_target)
  loss.backward()
  torch.nn.utils.clip_grad_norm(hash_model.parameters(), max_norm=15)
  optimizer.step()
  print(i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#@title Save model
for name, param in hash_model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
torch.save(hash_model.state_dict(), "/content/Model_1.pt")

fc1.weight tensor([[ 0.0253,  0.0189, -0.0075,  ..., -0.0139, -0.0050, -0.0204],
        [-0.0111, -0.0146, -0.0015,  ..., -0.0139, -0.0198, -0.0225],
        [-0.0228, -0.0017,  0.0257,  ..., -0.0220, -0.0287,  0.0083],
        ...,
        [ 0.0258, -0.0167,  0.0037,  ...,  0.0222,  0.0249, -0.0029],
        [-0.0050, -0.0087,  0.0219,  ..., -0.0029, -0.0099, -0.0126],
        [-0.0301,  0.0251, -0.0297,  ...,  0.0172,  0.0235,  0.0098]])
fc1.bias tensor([ 2.9605e-02, -1.4102e-02, -8.1195e-03,  2.6496e-02,  1.0909e-02,
        -2.6629e-02,  1.7401e-02, -1.7855e-02, -3.0480e-02, -5.8767e-03,
        -1.7524e-02,  3.2769e-03, -1.8845e-02,  6.6238e-03,  2.1777e-03,
        -2.9700e-02,  1.3434e-02, -1.9460e-02,  2.9687e-02, -1.9400e-02,
        -7.5495e-03, -1.1509e-03, -1.0167e-02, -7.1878e-04, -8.3346e-03,
        -4.4276e-03, -2.9212e-02,  2.9678e-02, -1.7935e-03, -2.1443e-02,
        -1.5937e-02,  5.2067e-03, -2.9545e-02,  2.6406e-02,  2.0758e-03,
        -2.3037e-02, -6.5784e-03,  

In [ ]:
#@title Convert to Binary
def convert_to_binary(vector):
  vector_tensor = torch.as_tensor(vector)
  sign = torch.sign(vector_tensor)
  binary_output = torch.relu(sign)  
  return binary_output


In [ ]:
#@title Postprocess Dimensionality Reduction
'''
Use dimensionality reduction
to reduce a bit_len x chunk_size 
matrix to a single bit_len vector 
while retaining as much of the information
from the digest as possible
'''
def reduce_digest(D): 
  # Convert tensy to a numpy vector
  D = D.detach().numpy()
  assert D.shape[0] % OUT_DIM == 0
  # Convert to bit_len x chunk_size numpy array
  temp = np.zeros((OUT_DIM, int(D.shape[0]/OUT_DIM)))
  for i in range(temp.shape[1]):
    for j in range(OUT_DIM):
      temp[j][i] = D[(i * OUT_DIM) + j]
  D = temp
  # Center the data 
  c = np.outer(np.ones(D.shape[1]), D.mean(0))
  D = D - c 
  # Get the top right singular vector
  U, S, Vt = LA.svd(D, full_matrices=True)
  Vt = Vt[:1, :].T  
  # Reconstruct the data
  D = D + c 
  # Project down to 1-dim
  return D @ Vt

In [ ]:
#@title Running Preprocess and Model output
collision = 0
hash_outputs = set()
gen = DocumentGenerator() 
for _ in range(10):
    send_file = gen.sentence()
    doc_feat = preprocess(send_file)
    print(doc_feat)
    hash_output = hash_model(doc_feat).detach().item()
    print(hash_output)
    '0x{0:08X}'.format(int(hash_output))
    # hash_output = torch.squeeze(hash_output.view(1,-1))
    # hash_output = torch.exp(hash_output)
    # if hash_output.shape[0] > OUT_DIM:
    #     hash_output = reduce_digest(hash_output)
    #     hash_output = torch.from_numpy(hash_output)
    print("Hash output: ", hash_output)
    # b = [" ".join(item) for item in binary_out.astype(str) ]
    # b = "".join(b)
    # bytes_hash = int(b, 2).to_bytes((len(b) + 7) // 8, 'big')
    # print(bytes_hash)
    if bytes_hash in hash_outputs:
      print(collision)
    else:
      hash_outputs.add(bytes_hash)
    collision +=1

Streaming output truncated to the last 5000 lines.
count:  63000
count:  64000
count:  65000
count:  66000
count:  67000
count:  68000
count:  69000
count:  70000
count:  71000
count:  72000
count:  73000
count:  74000
count:  75000
count:  76000
count:  77000
count:  78000
count:  79000
count:  80000
count:  81000
count:  82000
count:  83000
count:  84000
collision:  84932
count:  0
count:  1000
count:  2000
count:  3000
count:  4000
count:  5000
count:  6000
count:  7000
count:  8000
count:  9000
count:  10000
count:  11000
count:  12000
count:  13000
count:  14000
count:  15000
count:  16000
count:  17000
count:  18000
count:  19000
count:  20000
count:  21000
count:  22000
count:  23000
count:  24000
count:  25000
count:  26000
count:  27000
count:  28000
count:  29000
count:  30000
count:  31000
count:  32000
count:  33000
count:  34000
count:  35000
count:  36000
count:  37000
count:  38000
count:  39000
count:  40000
count:  41000
count:  42000
count:  43000
count:  44000
count:

In [ ]:
!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install inltk

In [ ]:
!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from inltk.inltk import setup
setup('en') 

In [ ]:
from inltk.inltk import get_similar_sentences

In [ ]:
# sample sentence
sentence1 = "The Quick Brown Fox Jumps Over The Lazy Dog"
sentence2 = "I eat rice"
sentence3 = "She sells seashells"
sentence4 = "We love icecream"
sentence5 = "Greatest Snow on Earth"
sentence6 = "We love the U"
# number of similar sentences to be generated
no_of_variants = 5                               # i want 6*5 similar sentences

# code of language 
code_of_language = "en"                                       # "en" is a code to represent english language

# where degree_of_aug is roughly the percentage of sentence you want to augment, with a default value of 0.1
degree_of_aug = 0.3

result1 = get_similar_sentences(sentence1, no_of_variants, code_of_language, degree_of_aug)
result2 = get_similar_sentences(sentence2, no_of_variants, code_of_language, degree_of_aug)
result3 = get_similar_sentences(sentence3, no_of_variants, code_of_language, degree_of_aug)
result4 = get_similar_sentences(sentence4, no_of_variants, code_of_language, degree_of_aug)
result5 = get_similar_sentences(sentence5, no_of_variants, code_of_language, degree_of_aug)
result6 = get_similar_sentences(sentence6, no_of_variants, code_of_language, degree_of_aug)


In [ ]:
for sentence1 in result1:
  print(sentence1)
for sentence2 in result2:
  print(sentence2)
for sentence3 in result3:
  print(sentence3)
for sentence4 in result4:
  print(sentence4)
for sentence5 in result5:
  print(sentence5)
for sentence6 in result6:
  print(sentence6)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/ltrc_yahoo (2).zip"

with ZipFile(file_name, 'r') as zipping:
  zipping.extractall("/content/Fair-PGRank/yahoo")
  print('Done')

Done


In [ ]:
from sklearn.datasets import load_svmlight_file

x_train, y_train, query_ids_train = load_svmlight_file("/content/Fair-PGRank/yahoo/ltrc_yahoo/set1.train.txt", query_id=True)
x_train = x_train.toarray()
x_train = np.unique(x_train, axis=0)
print(x_train.shape)

(438139, 699)
